In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# 데이터 불러오기
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    'foodornot',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',  # 이진 분류 (음식, 음식이 아님)
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'foodornot',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',  # 이진 분류 (음식, 음식이 아님)
    subset='validation'
)

# 모델 아키텍처 정의
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Early Stopping 콜백 정의
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# 모델 학습
history = model.fit(train_generator, validation_data=validation_generator, epochs=20, callbacks=[early_stopping])

# 새로운 이미지 예측
new_image_path = 'de.jpg'
new_image = tf.keras.preprocessing.image.load_img(new_image_path, target_size=(224, 224))
new_image = tf.keras.preprocessing.image.img_to_array(new_image)
new_image = new_image / 255.0
new_image = tf.expand_dims(new_image, axis=0)

prediction = model.predict(new_image)
if prediction[0] < 0.5:
    print('This is a food image.')
else:
    print('This is not a food image.')

Found 2268 images belonging to 2 classes.
Found 567 images belonging to 2 classes.
Epoch 1/20
71/71 [==============================] - 107s 1s/step - loss: 0.4578 - accuracy: 0.8302 - val_loss: 0.2704 - val_accuracy: 0.9065
Epoch 2/20
71/71 [==============================] - 120s 2s/step - loss: 0.2646 - accuracy: 0.8915 - val_loss: 0.2368 - val_accuracy: 0.8977
Epoch 3/20
71/71 [==============================] - 132s 2s/step - loss: 0.1739 - accuracy: 0.9321 - val_loss: 0.1230 - val_accuracy: 0.9559
Epoch 4/20
71/71 [==============================] - 130s 2s/step - loss: 0.1064 - accuracy: 0.9603 - val_loss: 0.1644 - val_accuracy: 0.9312
Epoch 5/20
71/71 [==============================] - 136s 2s/step - loss: 0.0539 - accuracy: 0.9824 - val_loss: 0.1321 - val_accuracy: 0.9612
Epoch 6/20
1/1 [==============================] - 0s 161ms/step
This is not a food image.


In [ ]:
# 저장된 가중치 불러오기
#model.load_weights("model_foodornot/weights.h5")

In [ ]:
# 디렉토리 생성
#if not os.path.exists("model_foodornot"):
#    os.makedirs("model_foodornot")

# 모델 가중치 저장
#model.save_weights("model_foodornot/weights.h5")

In [3]:
# 새로운 이미지 예측
new_image_path = 'tf3.jpg'
new_image = tf.keras.preprocessing.image.load_img(new_image_path, target_size=(224, 224))
new_image = tf.keras.preprocessing.image.img_to_array(new_image)
new_image = new_image / 255.0
new_image = tf.expand_dims(new_image, axis=0)

prediction = model.predict(new_image)
if prediction[0] < 0.5:
    print('This is a food image.')
else:
    print('This is not a food image.')

1/1 [==============================] - 0s 66ms/step
This is a food image.


In [4]:
# 새로운 이미지 예측
new_image_path = 'de.jpg'
new_image = tf.keras.preprocessing.image.load_img(new_image_path, target_size=(224, 224))
new_image = tf.keras.preprocessing.image.img_to_array(new_image)
new_image = new_image / 255.0
new_image = tf.expand_dims(new_image, axis=0)

prediction = model.predict(new_image)
if prediction[0] < 0.5:
    print('This is a food image.')
else:
    print('This is not a food image.')

1/1 [==============================] - 0s 60ms/step
This is not a food image.
